In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from string import digits

In [2]:
file = open("Assignment_4_data.txt","r")
labels = []
text = []
count = 0
for i in file.read().split('\n'):
    count+=1
    j = i.split("	")
    if(len(j)!=2):
        break
    labels.append(j[0])
    text.append(j[1]) 

In [3]:
ps = PorterStemmer()

In [4]:
def preprocess(text):
    #Break each sentence into tokens
    #Remove rem words
    #Use Porter Stemmer
    rem_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    vocab = []
    exceptions = ['.', ',', '?', '!', ':', ';', '\'t', "'s",')','(','"',']','[','\'',"#","*",'&','+','-','$','>','<','£']
    sw = stopwords.words("english")
    remove_digits = str.maketrans('', '', digits)
    for i in range(0,len(text)):
        text[i] = text[i].lower() 
        word_list = word_tokenize(text[i])
        final_word_list = []
        count = 0
        for j in range(0,len(word_list)):
            if(word_list[j] not in rem_words and word_list[j] not in sw):
                for chars in exceptions:
                    word_list[j] = word_list[j].replace(chars,"")
                word_list[j] = word_list[j].translate(remove_digits)
                word_list[j] = ps.stem(word_list[j])
                if(len(word_list[j])!=0):
                    final_word_list.append(word_list[j])
                    count +=1
                if(count >0 and final_word_list[count-1] not in vocab):
                    vocab.append(final_word_list[count-1])
        text[i] = final_word_list
    return vocab

In [5]:
vocabulary = preprocess(text)

In [6]:
def doc_vec(text,vocab):
    N = len(vocab)
    for i in range(0,len(text)):
        one_hot = np.zeros(N)
        for j in text[i]:
            indices = vocab.index(j)
            one_hot[indices] = 1
        text[i] = one_hot
    return text
        

In [8]:
text = doc_vec(text,vocabulary)
#print(text)

In [11]:
def data(text):
    value = list(range(0,len(text)))
    np.random.shuffle(value)
    last_index = int(0.8*len(value))
    train = (np.array(value[:last_index + 1])).reshape(-1,446)
    test = (np.array(value[last_index+1:])).reshape(1,-1)
    return train,test

In [12]:
train,test = data(text)
#print(len(train[0]))

In [19]:
input_dim = len(vocabulary)         # input dimension
hidden_1_dim = 100                  # hidden layer 1 dimension
output_dim = 1                      # output dimension
learning_rate = 1e-5                # Learning Rate


In [20]:
############## Random Initializations of the weights ##################
def weight_init():
    W1 = np.random.normal(-1,1,(input_dim,hidden_1_dim))
    W2 = np.random.normal(-1,1,(hidden_1_dim,output_dim))
    return W1,W2


In [30]:
def net_implement(x_batch,W1,W2,y,learning_rate,i_iter,loss_list):
    #Forward Pass
    a1 = x_batch @ W1
    h1 = np.maximum(0,a1)
    a2 = h1 @ W2
    h2 = 1/(1 + np.exp(-1*a2))
    
    #Loss Calculation
    loss = -1*(y*np.log(h2 + 0.0000025) + (1-y)*np.log(1-h2 + 0.0000025))
    
    
    #Gradient Calculation
    grad_a2 = y*(h2-1) + (1-y)*h2
    grad_W2 = np.matmul(h1.transpose(), grad_a2)
    grad_h1 = np.matmul(grad_a2,W2.transpose())
    grad_a1 = np.zeros(a1.shape)
    grad_a1[a1 > 0] = grad_h1[a1 > 0]
    grad_W1 = np.matmul(x_batch.transpose(),grad_a1)
    
    #Weight Updations
    W2 -= learning_rate * grad_W2
    W1 -= learning_rate * grad_W1
    
    return np.mean(loss)

In [31]:
def training_network(train,labels,text):
    no_of_iterations = 1000
    loss_list = []
    #Initialization of weights
    W1,W2 = weight_init()
    for i_iter in range(no_of_iterations):
        loss = 0
        for i in train:
            x_batch = []
            y_batch = []
            for index in i:
                x_batch.append(text[index])
                if(labels[index] == 'ham'):
                    y_batch.append(1)
                else:
                    y_batch.append(0)
            x_batch = (np.array(x_batch)).reshape(i.shape[0],-1)
            y_batch = (np.array(y_batch)).reshape(i.shape[0],-1)
            loss += net_implement(x_batch,W1,W2,y_batch,0.1,i_iter,loss_list)
        print(loss)
        loss_list.append(loss)
        return(W1,W2)

In [32]:
w1,w2 = training_network(train,labels,text)

8.03202770964219
